## ONNX推論測試

In [6]:
import numpy as np
import scipy.signal
import onnxruntime as rt
from scipy.io import wavfile
import IPython

# 確保音頻採樣率為指定值
def ensure_sample_rate(original_sample_rate, waveform, desired_sample_rate=16000):
    """Resample waveform if required."""
    if original_sample_rate != desired_sample_rate:
        desired_length = int(round(float(len(waveform)) /
                                   original_sample_rate * desired_sample_rate))
        waveform = scipy.signal.resample(waveform, desired_length)
    return desired_sample_rate, waveform

# 讀取並播放音檔
def load_and_play_audio(wav_file_name):
    sample_rate, wav_data = wavfile.read(wav_file_name, 'rb')
    sample_rate, wav_data = ensure_sample_rate(sample_rate, wav_data)
    waveform = wav_data / 32767
    IPython.display.display(IPython.display.Audio(wav_data, rate=sample_rate))
    return waveform

# 提取特徵
def extract_features(waveform, model_path):
    sess = rt.InferenceSession(model_path)
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[1].name
    pred_onx = sess.run([label_name], {input_name: np.array(waveform).astype('float32')})[0]
    return pred_onx

# 進行預測
def make_prediction(pred_onx, model_path, class_names):
    sess = rt.InferenceSession(model_path)
    input_name = sess.get_inputs()[0].name
    label_name = sess.get_outputs()[0].name
    onnx_pred = sess.run([label_name], {input_name: np.array(pred_onx)})[0]
    predicted_class_index = onnx_pred.mean(axis=0).argmax()
    predicted_class = class_names[predicted_class_index]
    return predicted_class

In [9]:
# 定義類別名稱
class_names = ['ON', 'OFF', 'FAN', 'BLADE FAULT', 'TAPPING']
# 音檔路徑
wav_file_name = 'dataset/test/FAN_GUARDS_20201210144646_6.wav'
# 讀取並播放音檔
waveform = load_and_play_audio(wav_file_name)
# 提取特徵
pred_onx = extract_features(waveform, "./model/model.onnx")
# 進行預測
predicted_class = make_prediction(pred_onx, "./model/tf_model.onnx", class_names)
print(predicted_class)

FAN


2024-01-28 16:46:33.876011604 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 48519, index: 1, mask: {2, 18, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-01-28 16:46:33.876057105 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 48518, index: 0, mask: {1, 17, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-01-28 16:46:33.942240581 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 48534, index: 1, mask: {2, 18, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-01-28 16:46:33.942254951 [E:onnxruntime:Default, env.cc:251 ThreadMain] pthread_setaffinity_np failed for thread: 48533, index: 0, mask: {1, 17, }, error code: 22 error msg: Invalid argument. Speci